# Multi-Pendulum Development

In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import odeint, solve_ivp
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import json
from MathFunctions import *
from DoublePendulumRefactor import DoublePendulum, DoublePendulumExplorer

In [2]:
# Declare variables & constants
t = sp.Symbol("t")
l1, l2, m1, m2, M1, M2, g = sp.symbols('l1 l2 m1 m2 M1 M2 g', real=True, positive=True)

# Declare functions
theta1 = sp.Function('theta1')(t)
theta2 = sp.Function('theta2')(t)
omega1 = sp.Function('omega1')(t)
omega2 = sp.Function('omega2')(t)

In [3]:
# Define parameters
params = {
    m1: 1,    # mass1
    m2: 1,    # mass2
    M1: 1,    # compound mass1
    M2: 1,    # compound mass2
    l1: 1,    # length1
    l2: 1,    # length2
    g : 9.81  # acceleration due to gravity
}

# using (time end - time start) * 200
# Is this enough/too much??
time = [0, 120, 48000]

In [4]:
# Instantiate DoublePendulumExplorer
simple_explorer = DoublePendulumExplorer(params, time, 'simple')
compound_explorer = DoublePendulumExplorer(params, time, 'compound')

----
&nbsp;
## Creating data dictionary and testing in Pandas

### Simple model

`Below Runtime approx: 2.5 mins`

In [9]:
simple_explorer.get_simulation_data(integrator=solve_ivp)

In [6]:
simple_dict = simple_explorer.simulation_data_dict
simple_df = pd.DataFrame(simple_dict[180])

print(simple_df.info())
simple_df.head()

#### Convert to JSON 

In [11]:
import json

# Function to convert NumPy arrays to lists
def numpy_to_list(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, dict):
        return {k: numpy_to_list(v) for k, v in data.items()}
    else:
        return data

In [ ]:
# Convert the simulation data to a JSON-serializable format
json_ready_data = {k: numpy_to_list(v) for k, v in simple_explorer.simulation_data_dict.items()}

# Serialize to a JSON file
with open('JSONdata/simple_data.json', 'w') as jsonfile:
    json.dump(json_ready_data, jsonfile, indent=4)

----
&nbsp;
### Compound model

In [5]:
compound_explorer.get_simulation_data(integrator=solve_ivp)

In [6]:
compound_dict = compound_explorer.simulation_data_dict
compound_df = pd.DataFrame(compound_dict[180])

print(compound_df.info())
compound_df.head()

#### Convert to JSON 

In [ ]:
# Convert the simulation data to a JSON-serializable format
json_ready_data = {k: numpy_to_list(v) for k, v in compound_explorer.simulation_data_dict.items()}

# Serialize to a JSON file
with open('JSONdata/compound_data.json', 'w') as jsonfile:
    json.dump(json_ready_data, jsonfile, indent=4)

----
&nbsp;
### Plot testing

`simple`

In [28]:
simple_explorer.find_poincare_section(y_fixed=-1.75, time_interval=(0, 120), angle_interval=(-180, 180))

In [29]:
simple_explorer.plot_poincare_map()

`compound`

In [26]:
compound_explorer.find_poincare_section(y_fixed=-1.75, time_interval=(0, 120), angle_interval=(-180, 180))

In [27]:
compound_explorer.plot_poincare_map()

### IDEA: 19/02/24

Record **points** $(x, y)$ that cross the plane rather than just focusing on the y-coordinate!

- If motion periodic, it was cross the fixed $yz$-plane at the same points every period. 
- If Lissajous curves then I'm not sure of the behaviour, nor how to record the point. The motion will be semi periodic. 
- If chaotic, then the $(x, y)$ crossing will appear almost random. 

#### STRATEGY

Rather than going for an enormous granular dataset, restrict it.
 - Less points, more $yz$ slices at regular intervals.
  - Glue together to give the portrait of the system as a whole. 


----
&nbsp;

### Current State of `DoublePendulumExplorer`

The `DoublePendulumExplorer` class extends a `DoublePendulum` class, focusing on exploring the dynamics of a double pendulum under various initial conditions, particularly varying `theta2` angles:

- **Running Simulations**: It can execute simulations across a predefined range of initial conditions.
- **Storing Positions**: Calculates and stores the positions $(x,y)$ of both pendulums for further analysis.


### Implemented Methods

- `_run_full_simulation_and_analysis`: Manages the full cycle of running simulations, calculating positions, and organizing the data.
- `_generate_initial_conditions`, `_run_simulations`, `_calculate_and_store_positions`: These methods collectively handle the simulation process.

We have added a class attribute, `get_simulation_data()` which calls all of the above and creates a dictionary `self.simulation_data_dict` which is indexed $\theta_2 \in [-\pi, \pi, \text{step}=0.5]$ $\implies$ each index represents $0.5 ^{\circ}$

The structure is:

```python
{0: {'theta1': array([ 0.00000000e+00,  6.00715702e-20,  2.40188874e-19, ...,
         -1.42244710e+00, -1.43566953e+00, -1.44593863e+00]),
  'theta2': array([-3.14159265, -3.14159265, -3.14159265, ..., 43.92106982,
         43.9840117 , 44.04642025]),
  'omega1': array([ 0.00000000e+00,  1.20118394e-17,  2.40061350e-17, ...,
         -1.47157868e+00, -1.17370579e+00, -8.80553174e-01]),
  'omega2': array([0.00000000e+00, 2.40256435e-17, 4.80278951e-17, ...,
         6.32831315e+00, 6.26317666e+00, 6.22103776e+00]),
  'x1': array([ 0.00000000e+00,  6.00715702e-20,  2.40188874e-19, ...,
         -9.89016419e-01, -9.90884258e-01, -9.92215399e-01]),
  'y1': array([-1.        , -1.        , -1.        , ..., -0.14780569,
         -0.13471595, -0.12453354]),
  'x2': array([-1.22464680e-16, -1.22404608e-16, -1.22224491e-16, ...,
         -1.05020550e+00, -9.89169705e-01, -9.28136239e-01]),
  'y2': array([ 0.        ,  0.        ,  0.        , ..., -1.14593189,
         -1.13471448, -1.12247836])}, 1: {..., }, 2: {..., } }
```

Although a good exercise is "How to" the two enormous 9GB datasets for `simple` and `compound` are way too granular. 
This dataset is unwieldy and will be **expensive** to host. 

See IDEA above for new strategy. 

The below are attempted implementations:

- `find_poincare_section`: Calculates the Poincaré section crossings.
- `plot_poincare_map`: Visualises the Poincaré section

Both require optimisation

### Outstanding Tasks

- **Poincaré Sections**: Identify and records the Poincaré section crossings based on a specific `y_fixed` value and within designated time and angle intervals.
- **Looping Trajectories**: Find them! that is, where $\theta_2>\pi$
- **Data Structure Optimization**: Explore database storage with MongoDB or something similar
- **Distinguishing Dense Regions**: Implementing a method to identify and visually distinguish dense regions of the Poincaré map, potentially indicating periodic orbits or other dynamic phenomena.
